In [1]:
## Query real time data from DingXiangYuan, and keep the latest records every day for each city

## Created on Sat Feb  8 12:41:50 2020
## Author: leebond
#### resource: https://github.com/jianxu305/nCov2019_analysis

import pandas as pd
import matplotlib.pyplot as plt
import utils   # some convenient functions
import pandas as pd
import pickle as pkl
import warnings
warnings.filterwarnings('ignore')
from googletrans import Translator # package used to translate Chinese into English
translator = Translator()

In [2]:
DXYArea = utils.load_chinese_data() # Query latest Regional Data from DXY
DXYArea.head(2)  

Latest Update Time:  2020-02-08 14:32:33.025000
Data Range:  2020-01-24 to 2020-02-08
Data Size:  24728


,provinceName,cityName,province_confirmedCount,province_suspectedCount,province_curedCount,province_deadCount,city_confirmedCount,city_suspectedCount,city_curedCount,city_deadCount,updateTime,updateDate
0,四川省,成都,363,0,54,1,109,0,33,1,2020-02-08 14:32:33.025,2020-02-08
1,四川省,南充,363,0,54,1,31,0,2,0,2020-02-08 14:32:33.025,2020-02-08


In [3]:
daily_frm_DXYArea = utils.aggDaily(DXYArea)  # aggregate to daily data (keep the latest records every day)

In [4]:
len(DXYArea)

24728

In [5]:
len(daily_frm_DXYArea)

5671

In [6]:
## Resoruce for Chinese - English Translation
with open('chineseProvince_to_EN.pkl','rb') as f:
    prov_dict = pkl.load(f)
with open('chineseCity_to_EN.pkl','rb') as f:
    city_dict = pkl.load(f)
    # some new area is not included in city_dict, so manually update
    city_dict['密云区'] = 'MiYun District' 
    city_dict['武清区'] = 'WuQing District'
    city_dict['兵团第十二师'] = '12-th BingTuanGroup'
    city_dict['昌吉州'] = 'ChangJi'
    city_dict['阿克苏地区'] = 'AKeSu'
        
def translate_to_English(data, prov_dict, city_dict):
    """Translate Chinese in dataset to English
    """        
    data['provinceName'] = data['provinceName'].apply(getProvinceTranslation)
    data['cityName'] = data['cityName'].apply(getCityTranslation)
    return data
    
def getProvinceTranslation(name):
    return prov_dict[name]

def getCityTranslation(name):
    try:
        return city_dict[name]
    except KeyError:
        print(name + 'cannot be translated, ask Yiran to mannully Translate\n')

In [7]:
daily_frm_DXYArea = translate_to_English(daily_frm_DXYArea, prov_dict, city_dict)

daily_frm_DXYArea.head(3)

,provinceName,cityName,confirmed,cured,dead,updateTime,updateDate
23738,Yunnan Province,Lijiang City,1,0,0,2020-01-24 23:55:14.973,2020-01-24
23736,Yunnan Province,Kunming,3,0,0,2020-01-24 23:55:14.973,2020-01-24
23737,Yunnan Province,Xishuangbanna,1,0,0,2020-01-24 23:55:14.973,2020-01-24


In [10]:
daily_frm_DXYArea.to_csv ('../data/DXYArea.csv', index = None, header=True)